In [12]:
from typing import Any, Dict, List, Union

from config2class.utils.filesystem import load_yaml

from typing import Any, Dict

from config2class.utils.dict_operations import find_cycles, flatten_dict, unflatten
from config2class.utils.token_operations import (
    build_dependency_graph,
    get_token_content,
    is_token,
    token_in,
)


def replace_tokens(d: Dict[str, Any]) -> Dict[str, Any]:
    pattern = r"\{\{.*?\}\}"

    if len(d) == 0:
        return d

    nested = len(d) > 1
    print(d)
    prefix = "__CONFIG__"
    if nested:
        d = {prefix: d}
    d_flatten = flatten_dict(d)

    # check incoming dict
    res = {}
    for key, value in d_flatten.items():
        if not is_token(value, pattern):
            res[key] = value
            continue

        token_value = get_token_content(value)
        split = token_value.split(".")
        split = [ele.strip() for ele in split]
        if nested and len(split[0]) == 0 and len(split) > 1:
            split[0] = prefix
        if "" in split or [" "] == split:
            raise ValueError(f"token={value} is not valid")
        
        value = "{{" + ".".join(split) + "}}"
        res[key] = value
    d_flatten = res

    dependencies = build_dependency_graph(d)
    cycles = len(find_cycles(dependencies)) > 0
    if cycles:
        raise ValueError(
            f"The config file contains cycles is therefor not valid to parse. Cycle: {cycles}"
        )

    while token_in(d_flatten):
        for key, value in d_flatten.items():
            if not is_token(value, pattern):
                continue
            # strip from token
            print(d_flatten)
            print(key)
            try:
                d_flatten[key] = d_flatten[value.strip("{}")]
            except KeyError as error:
                raise ValueError(f"token={value} is not valid") from error
    res = unflatten(d_flatten)
    if nested:
        _, res = res.popitem()
    return res


class ConfigAbstraction:
    """
    This class provides an abstraction for defining configuration data structures. It
    allows creating classes that represent configuration with typed fields and automatic
    generation of dataclasses with post-init logic for nested ConfigAbstraction objects.

    Attributes:
        name (str): The name of the configuration class.
        fields (Dict[str, Any]): A dictionary containing configuration fields and their values.
    """

    def __init__(self, name: str, fields: Dict[str, Any] = None):
        """
        Initializes a new ConfigAbstraction instance.

        Args:
            name (str): The name of the configuration class. This name will be converted
                to PascalCase (e.g., "my_config" becomes "MyConfig"). Leading underscores
                are treated specially, resulting in a single leading underscore in the
                generated class name (e.g., "_private_config" becomes "_PrivateConfig").
            fields (Dict[str, Any], optional): A dictionary containing configuration fields
                and their values. Defaults to None.
        """
        index = 0
        prefix = ""
        if name[0] == "_":
            index = 1
            prefix = "_"
        name = prefix + name[index].upper() + name[index + 1 :]
        self.name = name
        self.fields = fields if fields is not None else {}

    def add_field(
        self, key, value: Union[str, bool, float, list, tuple, int, "ConfigAbstraction"]
    ):
        """
        Adds a new field to the configuration structure.

        Args:
            key (str): The name of the field.
            value (Union[str, bool, float, list, tuple, int, ConfigAbstraction]): The value
                of the field. This can be a primitive type (str, bool, float, int), a
                list, a tuple, or another ConfigAbstraction instance.
        """
        self.fields[key] = value

    def write_code(self) -> List[str]:
        """
        Generates Python code for a dataclass representing the configuration structure.

        Returns:
            List[str]: A list of strings representing the generated Python code.
        """
        code = ["@dataclass\n", f"class {self.name}:\n"]
        post_init = {}
        for key, item in self.fields.items():
            if isinstance(item, ConfigAbstraction):
                typ = item.name
                post_init[key] = item
            else:
                typ = type(item).__name__
            code.append(f"    {key}: {typ}\n")

        # add class method
        code.append("\n    @classmethod")
        code.append(f'\n    def from_file(cls, file: str) -> "{self.name}":')
        code.append("\n        ending = file.split('.')[-1]")
        code.append("\n        content = getattr(fs_utils, f'load_{ending}')(file)")
        code.append("\n        content = replace_tokens(content)")
        code.append("\n        first_key, first_value = content.popitem()")
        code.append("\n        if len(content) == 0 and isinstance(first_value, dict):")
        code.append("\n            return cls(**first_value)")
        code.append("\n        else:")
        code.append("\n            content[first_key] = first_value")
        code.append("\n        return cls(**content)\n")

        # add post init func
        if len(post_init) == 0:
            return code

        code.append("\n    def __post_init__(self):\n")
        for key, value in post_init.items():
            code.append(
                f"        self.{key} = {value.name}(**self.{key})  #pylint: disable=E1134\n"
            )
        return code

# ==============================================================================================================================================

from typing import Any, Dict, List


class ConfigConstructor:
    """
    Constructs a Python dataclass from a nested dictionary representing configuration data.

    Attributes:
        configs (List[ConfigAbstraction]): A list of `ConfigAbstraction` instances,
            each representing a part of the configuration structure.
    """

    def __init__(self):
        """
        Initializes a new `ConfigConstructor` instance.
        """
        self.configs: List[ConfigAbstraction] = []

    def construct(self, config: Dict[str, Any]):
        """
        Parses the given configuration dictionary and constructs `ConfigAbstraction` instances
        to represent the configuration structure.

        Args:
            config (Dict[str, Any]): The configuration dictionary.
        """
        config = replace_tokens(config)
        if len(config) > 1:
            name = "Config"
            content = config
        else:
            name, content = list(config.items())[0]
        config_abstraction = self._construct_config_class(name, content)
        self.configs.append(config_abstraction)

    def write(self, out_path: str):
        """
        Writes the generated Python code to a file.

        Args:
            out_path (str): The path to the output file.
        """
        code = ["from dataclasses import dataclass\n"]
        code.append("import config2class.utils.filesystem as fs_utils\n\n\n")
        code.append("from config2class.utils.replacement import replace_tokens\n\n\n")
        
        for abstraction in self.configs:
            code.extend(abstraction.write_code())
            code.append("\n\n")

        code.pop(-1)
        with open(out_path, "w", encoding="utf-8") as file:
            file.writelines(code)

    def _construct_config_class(self, name: str, content: Dict[str, Any]):
        """
        Recursively constructs `ConfigAbstraction` instances for nested configurations.

        Args:
            name (str): The name of the configuration class.
            content (Dict[str, Any]): The configuration dictionary for this level.

        Returns:
            ConfigAbstraction: The constructed `ConfigAbstraction` instance.
        """
        config_abstraction = ConfigAbstraction(name, {})
        for key, value in content.items():
            if isinstance(value, dict) and len(value) > 0:
                sub_config = self._construct_config_class(name="_" + key, content=value)
                self.configs.append(sub_config)
                config_abstraction.add_field(key, sub_config)
            elif isinstance(value, (str, bool, float, list, tuple, int)):
                config_abstraction.add_field(key, value)

        return config_abstraction



content = load_yaml("example/example_token.yaml")
constructor = ConfigConstructor()
constructor.construct(content)
constructor.write("out.py")

print(content)

from out import App_config

{'app_config': {'test_value': '{{app_config.database.port}}', 'name': 'MyApp', 'version': '1.0.0', 'database': {'host': 'localhost', 'port': 5432, 'credentials': {'username': 'admin', 'password': 'secret'}}, 'features': {'authentication': True, 'caching': {'enabled': True, 'cache_size': 256}}}}
{'app_config.test_value': '{{app_config.database.port}}', 'app_config.name': 'MyApp', 'app_config.version': '1.0.0', 'app_config.database.host': 'localhost', 'app_config.database.port': 5432, 'app_config.database.credentials.username': 'admin', 'app_config.database.credentials.password': 'secret', 'app_config.features.authentication': True, 'app_config.features.caching.enabled': True, 'app_config.features.caching.cache_size': 256}
app_config.test_value
{'app_config': {'test_value': '{{app_config.database.port}}', 'name': 'MyApp', 'version': '1.0.0', 'database': {'host': 'localhost', 'port': 5432, 'credentials': {'username': 'admin', 'password': 'secret'}}, 'features': {'authentication': True, 'c

App_config(test_value=5432, name='MyApp', version='1.0.0', database=_Database(host='localhost', port=5432, credentials=_Credentials(username='admin', password='secret')), features=_Features(authentication=True, caching=_Caching(enabled=True, cache_size=256)))

In [83]:
import re


d = {
    "app_config": {
        "test_value": "{{app_config.name}}",
        "name": "MyApp",
        "version": "1.0.0",
        "database": {
            "host": "localhost",
            "port": 5432,
            "credentials": {"username": "{{app_config.test_value}}", "password": "secret"},
        },
        "features": {
            "authentication": True,
            "caching": {"enabled": True, "cache_size": 256},
        },
    }
}


def flatten_dict(d: dict):
    if d is None or len(d) == 0:
        return {}
    res = {}
    for key, value in d.items():
        if isinstance(value, dict):
            value = {key + "." + k: v for k, v in value.items()}
            res = {**res, **value}
            res = flatten_dict(res)
        else:
            res[key] = value
    return res

def find_cycles(graph):
    def visit(node, path, visited):
        if node in path:
            # Cycle detected; collect the cycle
            cycle_start_index = path.index(node)
            cycles.append(path[cycle_start_index:])
            return
        if node in visited:
            # Node already processed, skip
            return

        # Mark this node as visited and part of the current path
        visited.add(node)
        path.append(node)

        # Recurse to the next node if it exists
        if node in graph:
            next_node = graph[node]
            if next_node:  # There is an outgoing edge
                visit(next_node, path, visited)

        # Backtrack: remove the node from the current path
        path.pop()

    visited = set()
    cycles = []

    # Check each node in the graph
    for node in graph:
        if node not in visited:
            visit(node, [], visited)
    
    return cycles

# find nodes with token
token = "{{_}}"

def get_token_content(value: str) -> str:
    return value.strip("{}")

def is_token(value, pattern):
    return isinstance(value, str) and re.search(pattern, str(value))


def build_dependency_graph(d: dict, pattern: str = r"\{\{.*?\}\}"):
    d_flatten = flatten_dict(d)
    dependencies = {}
    for key, value in d_flatten.items():
        if not is_token(value, pattern):
            continue
        dependencies[key] = value.strip("{}")
    return dependencies

def token_in(d: dict[str, Any]) -> bool:
    for value in d.values():
        if is_token(value, pattern):
            return True
    return False
dependencies = build_dependency_graph(d)
cycles = len(find_cycles(dependencies)) > 0 

if cycles:
    raise ValueError("The config file contains cycles is therefor not valid to parse")

pattern: str = r"\{\{.*?\}\}"
# replace dependencies 
# let the information flow from  leaves to roots
d_flatten = flatten_dict(d)


while token_in(d_flatten):
    for key, value in d_flatten.items():
        if not is_token(value, pattern):
            continue
        # strip from token
        d_flatten[key] = d_flatten[value.strip("{}")] 

# rebuild config
def unflatten(dictionary):
    # from https://stackoverflow.com/questions/6037503/python-unflatten-dict
    resultDict = dict()
    for key, value in dictionary.items():
        parts = key.split(".")
        d = resultDict
        for part in parts[:-1]:
            if part not in d:
                d[part] = dict()
            d = d[part]
        d[parts[-1]] = value
    return resultDict

unflatten(d_flatten)


{'app_config': {'test_value': 'MyApp',
  'name': 'MyApp',
  'version': '1.0.0',
  'database': {'host': 'localhost',
   'port': 5432,
   'credentials': {'username': 'MyApp', 'password': 'secret'}},
  'features': {'authentication': True,
   'caching': {'enabled': True, 'cache_size': 256}}}}

In [98]:
d = {
    "app_config": {
        "test_value": "{{.name}}",
        "name": "MyApp",
        "version": "1.0.0",
        "database": {
            "host": "localhost",
            "port": 5432,
            "credentials": {"username": "{{.test_value}}", "password": "secret"},
        },
        "features": {
            "authentication": True,
            "caching": {"enabled": True, "cache_size": 256},
        },
    }
}

def replace_embedded_token(d: Dict[str, Any]) -> Dict[str, Any]:
    pattern = r"\{\{.*?\}\}"
    
    if len(d) == 0:
        return d
    
    nested = len(d) > 1
    prefix = "__CONFIG__"
    if nested:
        d = {prefix: d}
    d_flatten = flatten_dict(d)
    

    # check incoming dict
    res = {}
    for key, value in d_flatten.items():
        if not is_token(value, pattern):
            res[key] = value
            continue
        
        token_value = get_token_content(value)
        split = token_value.split(".")
        if nested and split[0] == "":
            split[0] = prefix
        
        if "" in split:
            raise ValueError(f"token={value} is not valid")
        value = "{{" + ".".join(split) + "}}"
        res[key] = value
    d_flatten = res

    dependencies = build_dependency_graph(d)
    cycles = len(find_cycles(dependencies)) > 0 
    if cycles:
        raise ValueError("The config file contains cycles is therefor not valid to parse")

    while token_in(d_flatten):
        for key, value in d_flatten.items():
            if not is_token(value, pattern):
                continue
            # strip from token
            try:
                d_flatten[key] = d_flatten[value.strip("{}")] 
            except KeyError as error:
                raise ValueError(f"token={value} is not valid")                
    res = unflatten(d_flatten)
    if nested:
        _, res = res.popitem()
    return res
replace_embedded_token(d["app_config"])

{'test_value': 'MyApp',
 'name': 'MyApp',
 'version': '1.0.0',
 'database': {'host': 'localhost',
  'port': 5432,
  'credentials': {'username': 'MyApp', 'password': 'secret'}},
 'features': {'authentication': True,
  'caching': {'enabled': True, 'cache_size': 256}}}